**Authors**: Malthe D. Andersen (s194257), Yucheng Fu (s194241), Andreas Høst (s194238)

In [ ]:
#packages

# Scraping the data

# Network

## Creating the network

## Graphing

## Importance

# Sentiment analysis